In [1]:
# Manipulation de données 
import pandas as pd 
import numpy as np



# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from PIL import Image



# Preprocessing des reviews 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

    # Pour filtrer la langue des avis
from langdetect import detect, DetectorFactory 
from langdetect.lang_detect_exception import LangDetectException

# CHARGEMENT DES DONNEES

In [2]:
# Initialisation d'un dataframe vide
df = pd.DataFrame()

# Boucle pour ouvrir et concaténer les 10 fichiers
for i in range(1,11):
    df_to_concat = pd.read_csv(f'Data/avis_ugc_part{i}.csv')
    df = pd.concat([df, df_to_concat])

In [3]:
df.head()

,rating,date,review,likes,user_name,user_nb_reviews,user_nb_photos,cinéma,language
0,1,2024-11-10T07:41:44.880Z,Il n’a pas admissible qu’une grosse chaîne de ...,0,anne degroux,2,0,UGC Ciné Cité Les Halles,fr
1,4,2024-11-09T10:32:32.722Z,"Pratique, plein de salles et bonne programmati...",0,Géraldine Sauvenay,3,0,UGC Ciné Cité Les Halles,fr
2,3,2024-11-09T08:45:17.039Z,"Nettoyez vos écrans, ça commence à se voir !",0,M G,2,0,UGC Ciné Cité Les Halles,fr
3,1,2024-11-08T20:13:38.816Z,The theater still has a bed bug problem. Sat d...,0,Cordelia Ryan,2,1,UGC Ciné Cité Les Halles,fr
4,1,2024-11-08T15:32:14.074Z,VOLUME beaucoup trop fort mon dieu!! On était ...,0,Spei,83,0,UGC Ciné Cité Les Halles,fr


# PREPROCESSING

Avant de passer nos données dans les modèles, nous devons les mettre en forme. Tout d’abord, j’ai constaté que la colonne `language` contient souvent des informations incorrectes sur la langue. Pour remédier à cela, nous allons utiliser la librairie `langdetect`, qui s’appuie sur la détection de langue de Google pour identifier la langue correcte.

In [ ]:
"""df_clean = df.copy()"""

# Importation du fichier déjà nettoyé car le code est long à tourner
df_clean = pd.read_csv('Data/avis_ugc_fr.csv')

In [ ]:
"""DetectorFactory.seed = 24

# Fonction pour détecter la langue d'un texte donné
def detect_language(text: str) -> str | None:
    try:
        # Tente de détecter la langue du texte
        return detect(text)
    except LangDetectException:
        # En cas d'erreur (par exemple, si le texte est vide ou ambigu), retourne None
        return None

# Applique la fonction de détection de langue à chaque avis dans la colonne 'review'
df_clean['language_detected'] = df_clean['review'].map(detect_language)

# Filtre les données pour ne conserver que les avis détectés en français ('fr')
df_clean = df[df['language_detected'] == 'fr']

# Exporte le DataFrame filtré dans un fichier CSV
df_clean.to_csv('Data/avis_ugc_fr.csv', index=False)"""

In [ ]:
print(f" Nombre d'avis non francais supprimés : {df.shape[0] - df_clean.shape[0]}")

In [ ]:
# Importation des stop words français de la librairie NLTK
stop_words = set(stopwords.words('french'))

# Ajout de stop words détectés dans les avis 
custom_stop_words = {",", ".", "a", "c'est", "!", "film", "cinéma", 
                     "si", "plus", "ugc", "ca", "là", "où", "deux", 
                     "ça", "h", "qu", "il", "''", '``', "-", "(", ")",
                     "..", "€", "?", "....", "//", ":", "/", ";", '’'
                    }
stop_words.update(custom_stop_words)

# Initialisation d'un lemmatiseur pour réduire les mots à leur forme de base
lemmatizer = WordNetLemmatizer()

def preprocess_reviews(reviews: pd.Series) -> pd.Series:
    """
    Cette fonction prend une série pandas contenant des avis textuels et applique un prétraitement
    pour chaque avis. Elle effectue les étapes suivantes :
    
    1. Convertit chaque avis en minuscules pour uniformiser le texte.
    2. Supprime les chiffres et remplace les apostrophes par des espaces pour éviter les séparations incorrectes.
    3. Tokenise chaque avis, c'est-à-dire qu'elle divise le texte en mots individuels.
    4. Supprime les stop words (mots courants sans importance) à partir d'une liste de stop words standard en français,
       enrichie de mots spécifiques au contexte.
    5. Applique la lemmatisation, qui réduit les mots à leur forme de base (par exemple, "films" devient "film").
    
    Paramètres :
    reviews (pd.Series) : Série pandas contenant les avis textuels.
    
    Retourne :
    pd.Series : Série pandas contenant des listes de mots prétraités pour chaque avis.
    """    
    
    # Convertir en minuscules et supprimer les chiffres et les apostrophes
    reviews = reviews.str.lower().str.replace(r'\d+', '', regex=True).str.replace("'", ' ')

    # Appliquer tokenisation, suppression des stop words, et lemmatisation
    reviews = reviews.apply(lambda review: [lemmatizer.lemmatize(word) 
                                            for word in word_tokenize(review) 
                                            if word not in stop_words
                                           ])


    return reviews

In [ ]:
reviews = preprocess_reviews(df_clean['review'])

In [ ]:
def my_word_cloud(reviews: pd.Series, title: str = None, subplot: tuple[int] = None, background_image_path: str = None) -> None:
    """
    Génère et affiche un nuage de mots basé sur le contenu textuel d'une série pandas contenant des avis.
    La fonction peut également afficher le nuage de mots sur une image de fond spécifiée.
    
    Paramètres :
    - reviews (pd.Series) : Série pandas contenant les avis textuels. Tous les avis sont concaténés pour créer le nuage de mots.
    - title (str, optionnel) : Titre à afficher au-dessus du nuage de mots. Si None, aucun titre n'est affiché.
    - subplot (tuple[int], optionnel) : Spécifie l'emplacement du nuage de mots dans une grille de sous-plots sous la forme (nrows, ncols, index).
      Utilisé lorsque le nuage de mots doit être affiché dans une figure avec plusieurs sous-graphiques.
    - background_image_path (str, optionnel) : Chemin d'accès à une image de fond pour le nuage de mots.
      Si fourni, le nuage de mots prendra la forme de cette image. Si None, le fond sera blanc par défaut.
    
    Retourne :
    - None : La fonction affiche directement le nuage de mots en utilisant `matplotlib.pyplot` et ne retourne rien.
    """
    
    # Concatène tous les avis en une seule chaîne de texte
    text = ' '.join(reviews.astype(str))
    
    # Charge l'image de fond si spécifiée, sinon crée un nuage de mots avec un fond blanc
    if background_image_path is not None:
        background_image = np.array(Image.open(background_image_path))
        wordcloud = WordCloud(width=800, height=400, background_color='white', mask=background_image, contour_width=1, contour_color='black').generate(text)
    else:
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    

    plt.figure(figsize=(10, 5))
    
    # Positionne le nuage de mots dans un sous-plot si `subplot` est spécifié
    if subplot is not None :
        plt.subplot(subplot[0], subplot[1], subplot[2])
    
    # Affiche le nuage de mots et enlève les axes
    plt.imshow(wordcloud)
    plt.axis('off')
    
    # Ajoute un titre si spécifié
    if title is not None:
        plt.title(title)
    
    plt.show()

In [ ]:
my_word_cloud(reviews)

In [ ]:
# Boucle pour afficher un word cloud pour chaque note
for i in np.sort(df_clean['rating'].unique()):
    reviews_per_note =  preprocess_reviews(df_clean[df_clean['rating'] == i]['review'])
    
    title = f' Word Cloud des avis {i} étoiles'
    my_word_cloud(reviews_per_note, title, subplot=(3,2,i))

* Le mot "salle(s)" apparaît aussi bien dans les avis positifs que négatifs, ce qui suggère qu'il n'est pas un indicateur fiable de sentiment. Il serait donc pertinent de le retirer, au moins pour la création du nuage de mots, afin d'obtenir une visualisation plus représentative des termes associés aux avis positifs ou négatifs.

In [4]:
# Boucle pour afficher un word cloud pour chaque note
for i in np.sort(df_clean['rating'].unique()):
    reviews_per_note =  preprocess_reviews(df_clean[df_clean['rating'] == i]['review'])
    
    reviews_per_note = reviews_per_note.apply(lambda words: [word for word in words if word not in ['salle', 'salles']])
    
    title = f' Word Cloud des avis {i} étoiles'
    my_word_cloud(reviews_per_note, title, subplot=(3,2,i))

NameError: name 'df_clean' is not defined

1. **Word Cloud des avis 1 étoile :**

    * Les mots les plus fréquents incluent "place", "séance", et "bien". Cela pourrait suggérer des commentaires sur l'emplacement, l'organisation des séances, ou même des aspects de confort. Cependant, dans un contexte de note basse, ces termes sont probablement utilisés de manière négative (ex. : mauvaise organisation des séances, inconfort des places).
    * Les termes "dommage", "problème", et "mal" apparaissent également, ce qui renforce l'idée de critiques.

2. **Word Cloud des avis 2 étoiles :**

    * On retrouve des mots similaires comme "place" et "tout". "Dommage" est également présent, ce qui indique des critiques mitigées.
    * Le mot "bien" reste fréquent, mais dans ce contexte, il pourrait être utilisé pour souligner des aspects corrects dans un avis globalement négatif.

3. **Word Cloud des avis 3 étoiles :**

    * Les termes "bien" et "peu" sont dominants, suggérant des avis modérés où des aspects positifs sont soulignés, mais avec des réserves (ex. : "peu d'espace", "bien mais pourrait être mieux").
    * D’autres mots comme "dommage", "prix", et "petite" indiquent probablement des points d'amélioration signalés par les utilisateurs.

4. **Word Cloud des avis 4 étoiles :**

    * Le mot "bien" devient encore plus dominant, accompagné de "très", "bon", et "propre". Cela suggère des avis majoritairement positifs, où la qualité et le confort sont reconnus.
    * Des termes comme "agréable" et "choix" apparaissent, ce qui peut indiquer une satisfaction avec les options disponibles et l’atmosphère générale.

5. **Word Cloud des avis 5 étoiles :**

    * Les mots les plus fréquents sont "très", "super", "bien", et "agréable", soulignant une satisfaction élevée.
    * On voit aussi "parfait", "personnel", et "choix", ce qui indique une expérience globalement positive avec de nombreux aspects appréciés, y compris le service.